In [28]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import math
from datetime import datetime
import time
import IPython
import os
import threading
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('error', category=UserWarning)

In [29]:
REGIONLIST= ["US", "EU", "UK"]
MODELLIST = ["64", "256", "512"]
ESTIMATELIST = ["Q1","Q2", "Q3"]

In [30]:
def AllPossibleQueues_Func():
  result = []
  for r in REGIONLIST:
    if (r == "ALL"):
      continue
    for m in MODELLIST:
      for e in ESTIMATELIST:
        result.append((r,m,e))
    
  return result

ALLPOSSIBLEQUEUES = AllPossibleQueues_Func()
print(ALLPOSSIBLEQUEUES)

[('US', '64', 'Q1'), ('US', '64', 'Q2'), ('US', '64', 'Q3'), ('US', '256', 'Q1'), ('US', '256', 'Q2'), ('US', '256', 'Q3'), ('US', '512', 'Q1'), ('US', '512', 'Q2'), ('US', '512', 'Q3'), ('EU', '64', 'Q1'), ('EU', '64', 'Q2'), ('EU', '64', 'Q3'), ('EU', '256', 'Q1'), ('EU', '256', 'Q2'), ('EU', '256', 'Q3'), ('EU', '512', 'Q1'), ('EU', '512', 'Q2'), ('EU', '512', 'Q3'), ('UK', '64', 'Q1'), ('UK', '64', 'Q2'), ('UK', '64', 'Q3'), ('UK', '256', 'Q1'), ('UK', '256', 'Q2'), ('UK', '256', 'Q3'), ('UK', '512', 'Q1'), ('UK', '512', 'Q2'), ('UK', '512', 'Q3')]


In [31]:
data = pd.read_excel('stds.xlsx')

In [32]:
data.head()

,Timestamp,Region,Model,rtReserveTime or preorder-email time,Initial Valve Estimate,Your most recent pre-order estimated time,When did you receive your ready to order email?,Unnamed: 7
0,2022-02-25 13:21:39.764,US,256,1626454969,December,Q1,1.645813e+09,NaN
1,2022-02-09 14:50:45.171,UK,512,1635594996,After Q2,After Q2,NaN,NaN
2,2022-02-09 14:54:01.852,US,512,1626454856,December,Q1,NaN,NaN
3,2022-02-09 14:58:39.188,US,256,1644440188,Q1,Q2,NaN,NaN
4,2022-02-09 15:04:48.617,EU,256,1626478685,Q1,Q2,NaN,NaN


In [33]:
data.drop(columns=["Timestamp", "Initial Valve Estimate", "Unnamed: 7"], inplace=True)
data.rename(columns = {"rtReserveTime or preorder-email time ": "Timestamp", "Your most recent pre-order estimated time":"ValveEstimate", "When did you receive your ready to order email?":"TrueOrderTimestamp"}, inplace=True)


namask = data["Region"].isna() | data["Model"].isna() | data["ValveEstimate"].isna() | data["Timestamp"].isna() | data["ValveEstimate"].isna()
print(f"{np.sum(namask)} entries removed for having NA on vital fields\n")
data.drop(data[namask].index,inplace=True)

13 entries removed for having NA on vital fields



In [34]:
#not valid for after Q2
#data = data[data["ValveEstimate"] != "After Q2"]
notEstimable = np.vectorize(lambda x: x not in ESTIMATELIST)
notEstimableMask = notEstimable(data["ValveEstimate"])
print(f"{np.sum(notEstimableMask)} entries removed for being \"After Q2\", \"After Q3\", or similar\n")
data.drop(data[notEstimableMask].index, inplace=True)

764 entries removed for being "After Q2", "After Q3", or similar



In [35]:
#entries supposedly happening after feb 25th 10am PDT
wrongUpdateMask = ((data["ValveEstimate"] == "Q1") | (data["ValveEstimate"] == "Q2") | (data["ValveEstimate"] == "Q3")) & (data["Timestamp"] >= 1645801200)
print(f"{np.sum(wrongUpdateMask)} entries removed for being after Februrary 25th 10am PDT\n")
data.drop(data[wrongUpdateMask].index, inplace=True)

51 entries removed for being after Februrary 25th 10am PDT



In [36]:
 #entries supposedly happening before july 16th 10am PDT
cretinMask = data["Timestamp"] < 1626454800
print(f"{np.sum(cretinMask)} entries removed for being before July 16th 10am PDT\n")
data.drop(data[cretinMask].index, inplace=True)

0 entries removed for being before July 16th 10am PDT



In [37]:
data.head()

,Region,Model,Timestamp,ValveEstimate,TrueOrderTimestamp
0,US,256,1626454969,Q1,1.645813e+09
2,US,512,1626454856,Q1,NaN
3,US,256,1644440188,Q2,NaN
4,EU,256,1626478685,Q2,NaN
5,US,512,1627423926,Q3,NaN


In [38]:
  @np.vectorize
  def digits(x):
    if (np.isnan(x)):
      return -1
    
    return int(np.log10(x))+1
  
  @np.vectorize
  def convertTimestampToSeconds(x):
    if (np.isnan(x)):
      return np.nan
    
    l = digits(x)
    if (l > 10 and (l - 10) % 3 == 0):
      m = (l - 10) // 3
      for _ in range(m):
        x = x // 1000
    
    return x

In [39]:
data["Timestamp"] = convertTimestampToSeconds(data["Timestamp"])
data["TrueOrderTimestamp"] = convertTimestampToSeconds(data["TrueOrderTimestamp"])

In [40]:
data.to_csv("Data.csv")

In [41]:
removeEmpty = data["TrueOrderTimestamp"].isna()
print(f"{np.sum(removeEmpty)} remove empty entries")
data.drop(data[removeEmpty].index, inplace=True)

2032 remove empty entries


In [43]:
trainData = data.copy()

In [44]:
trainData.dtype

In [45]:
s = (trainData.dtypes == 'object')
object_cols = list(s[s].index)

In [46]:
trainData5158

,Region,Model,Timestamp,ValveEstimate,TrueOrderTimestamp
0,US,256,1626454969,Q1,1.645813e+09
6,EU,512,1626454854,Q1,1.645813e+09
34,UK,256,1626455083,Q1,1.646675e+09
63,EU,64,1626464450,Q1,1.647279e+09
101,EU,512,1626455384,Q1,1.646675e+09
...,...,...,...,...,...
2587,US,512,1626455224,Q1,1.647881e+09
2588,US,256,1626455468,Q1,1.647880e+09
2616,EU,256,1626463800,Q1,1.647881e+09
2628,US,256,1626455340,Q1,1.647971e+09
